We observed a vulnerability in AES when used in ECB mode. We decided to modify the algorithm to avoid duplicate encrypted blocks resulting from encrypting identical blocks in the plaintext. The code for the AES-ADD version used to encrypt images in PPM P6 format is available:

In [ ]:
%pip install pycryptodome

In [ ]:
import math
import os

from Crypto.Cipher import AES

BLOCK_SIZE = 16
UMAX = int(math.pow(256, BLOCK_SIZE))


def num_to_bytes(n: int) -> bytes:
    length = (n.bit_length() + 7) // 8
    padding = length % BLOCK_SIZE
    if padding != 0:
        length += BLOCK_SIZE - padding
    return n.to_bytes(length, "big")


def bytes_to_num(b: bytes) -> int:
    return int(b.hex(), 16)


def parse_header_ppm(data: bytes) -> tuple[bytes, bytes]:
    header_len = 0
    header_lines = 0
    inside_comment = False

    for header_len, b in enumerate(data):
        if inside_comment and b == ord("\n"):
            inside_comment = False
            continue
        if b == ord("#"):
            inside_comment = True
            continue
        if b == ord("\n"):
            # counts only non-comment lines
            header_lines += 1
        if header_lines == 3:
            # include the current '\n' character in the header
            header_len += 1
            break

    return data[:header_len], data[header_len:]


def pad(plaintext: bytes) -> bytes:
    padding = BLOCK_SIZE - len(plaintext) % BLOCK_SIZE
    return plaintext + bytes(chr(padding), encoding="utf8") * padding


def aes_add_encrypt(plaintext: bytes, key: bytes) -> bytes:
    cipher = AES.new(key, AES.MODE_ECB)
    ciphertext = cipher.encrypt(pad(plaintext))
    blocks = [
        ciphertext[i * BLOCK_SIZE : (i + 1) * BLOCK_SIZE]
        for i in range(len(ciphertext) // BLOCK_SIZE)
    ]

    iv = os.urandom(BLOCK_SIZE)
    blocks.insert(0, iv)

    for i in range(len(blocks) - 1):
        prev_block = bytes_to_num(blocks[i])
        curr_block = bytes_to_num(blocks[i + 1])

        n_curr_block = (prev_block + curr_block) % UMAX
        blocks[i + 1] = num_to_bytes(n_curr_block)

    ciphertext_add = b"".join(blocks)

    return ciphertext_add


def aes_add_encrypt_file(
    input_filename: str,
    output_filename: str,
    key: bytes,
) -> None:
    with open(input_filename, "rb") as f:
        header, data = parse_header_ppm(f.read())

    c_text = aes_add_encrypt(data, key)

    with open(output_filename, "wb") as fw:
        fw.write(header)
        fw.write(c_text)

# KEY = b"ttxgWJEjvRReRIyrlKJaFtocIvDyQqni"
# INPUT = "my_precious.ppm"
# OUTPUT = "my_precious.enc.ppm"

# aes_add_encrypt_file(input_filename=INPUT, output_filename=OUTPUT, key=KEY)

The encryption algorithm still employs the AES-ECB variant but introduces a new operation on the ciphertext blocks. In this variant, the algorithm follows the idea of cipher block chaining (CBC), as illustrated in the figure below:
![CBC](https://upload.wikimedia.org/wikipedia/commons/thumb/8/80/CBC_encryption.svg/1200px-CBC_encryption.svg.png)
In the AES-ADD variant, each block is modified by adding it to the previous block, as evident in the encryption function.
![AES](https://i.stack.imgur.com/bXAUL.png)

In [ ]:
from collections import Counter

from PIL import Image


def get_image_size(header: bytes) -> tuple[int, int]:
    dheader: str = header.decode("utf-8")

    # ignore comments as well
    lines = [line for line in dheader.split("\n") if not line.startswith("#")]

    # PPM format:
    #   - line 1 -> P6
    #   - line 2 -> width height
    #   - line 3 -> maximum pixel intensity
    width, height = map(int, lines[1].split())

    return width, height


def break_encryption(
    ecb_blocks: list[int], w: int, h: int, background: int
) -> Image.Image:
    im = Image.new("RGB", (w, h))
    data = []
    for b in ecb_blocks:
        if b == background:
            data += [(0, 0, 0)]
        else:
            data += [(255, 255, 255)]
    im.putdata(data)
    return im


def decrypt_image(enc_filename: str) -> Image.Image:
    with open(enc_filename, "rb") as file:
        header, data = parse_header_ppm(file.read())

    width, height = get_image_size(header)

    ########## SOLUȚIA AICI ##########
    # TODO 1: Inversați pasul ADD din AES-ADD.
    def inverse_addition(block1, block2):
        # Implementați inversa adunării modulo UMAX
        return (block1 - block2) % UMAX

    # Citim blocurile de la începutul datelor
    blocks = [
        data[i * BLOCK_SIZE : (i + 1) * BLOCK_SIZE]
        for i in range(len(data) // BLOCK_SIZE)
    ]

    # Inițializăm cu blocul IV (primul bloc)
    ecb_blocks = [bytes_to_num(blocks[0])]

    # Aplicăm inversarea pasului ADD
    for i in range(1, len(blocks)):
        prev_block = bytes_to_num(blocks[i - 1])
        curr_block = bytes_to_num(blocks[i])

        n_curr_block = inverse_addition(curr_block, prev_block)
        ecb_blocks.append(n_curr_block)

    # TODO 2: Determinati block-ul din cipertext-ul AES-ECB care se repetă de
    # cele mai multe ori. Acesta va fi culoarea de background, în imaginea
    # alb-negru pe care o veti obtine.
    background = Counter(ecb_blocks).most_common(1)[0][0]
    ##################################

    # Using the scale factor, we make the image more readable. Also, the image
    # will repeat `scale_factor` times.
    scale_factor = 2
    img = break_encryption(
        ecb_blocks,
        round(scale_factor * 3 * width / 16),
        height // scale_factor + 1,
        background,
    )
    return img


im = decrypt_image("my_precious.enc.ppm")
im